Notebook for testing docking against Mpro

In [1]:
from ccdc.docking import Docker
from ccdc.io import MoleculeReader, EntryReader

docker = Docker()
settings = docker.settings

MLL1_protein_file = '/Applications/CCDC/Python_API_2022/docs/example_files/2w5y_protein_prepared.mol2'
settings.add_protein_file(MLL1_protein_file)

MLL1_native_ligand_file = '/Applications/CCDC/Python_API_2022/docs/example_files/SAH_native.mol2'
MLL1_native_ligand = MoleculeReader(MLL1_native_ligand_file)[0]
MLL1_protein = settings.proteins[0]
settings.binding_site = settings.BindingSiteFromLigand(MLL1_protein, MLL1_native_ligand, 8.0)

settings.fitness_function = 'plp'
settings.autoscale = 10.
settings.early_termination = False
import tempfile
batch_tempd = tempfile.mkdtemp()
settings.output_directory = batch_tempd
settings.output_file = '/Applications/CCDC/Python_API_2022/docs/example_files/docked_ligands.mol2'

MLL1_ligand_file = '/Applications/CCDC/Python_API_2022/docs/example_files/SAH.mol2'
MLL1_ligand = MoleculeReader(MLL1_ligand_file)[0]
settings.add_ligand_file(MLL1_ligand_file, 10)

results = docker.dock() 

Starting GOLD with conf file /Users/williammccorkindale/ml_physics/smi2wyck/notebooks/docking/api_gold.conf
Setting up GOLD environment...
GOLD Version 2022.1.0
Running:
 
     "/Applications/CCDC/Discovery_2022/GOLD/gold/d_macx/bin/gold_macx" "/Users/williammccorkindale/ml_physics/smi2wyck/notebooks/docking/api_gold.conf"



Mpro

In [3]:
import sys
from prody import *
from rdkit import Chem
from rdkit.Chem import AllChem
from io import StringIO
import pypdb


def describe_chemical(chem_id):
    """

    Parameters
    ----------

    chem_id : string
        A 4 character string representing the full chemical sequence of interest (ie, NAG)

    Returns
    -------

    out : dict
        A dictionary containing the chemical description associated with the PDB ID

    Examples
    --------
    >>> chem_desc = describe_chemical('NAG')
    >>> print(chem_desc)
    {'describeHet': {'ligandInfo': {'ligand': {'@molecularWeight': '221.208',
    'InChIKey': 'OVRNDRQMDRJTHS-FMDGEEDCSA-N', '@type': 'D-saccharide',
    'chemicalName': 'N-ACETYL-D-GLUCOSAMINE', '@chemicalID': 'NAG',
    'smiles': 'CC(=O)N[C@@H]1[C@H]([C@@H]([C@H](O[C@H]1O)CO)O)O', '
    InChI': 'InChI=1S/C8H15NO6/c1-3(11)9-5-7(13)6(12)4(2-10)15-8(5)14/
    h4-8,10,12-14H,2H2,1H3,(H,9,11)/t4-,5-,6-,7-,8-/m1/s1',
    'formula': 'C8 H15 N O6'}}}}

    """
    out = pypdb.get_info(chem_id, url_root = 'http://www.rcsb.org/pdb/rest/describeHet?chemicalID=')
    out = pypdb.to_dict(out)
    return out

def get_pdb_components(pdb_id):
    """
    Split a protein-ligand pdb into protein and ligand components
    :param pdb_id:
    :return:
    """
    pdb = parsePDB(pdb_id)
    protein = pdb.select('protein')
    ligand = pdb.select('not protein and not water')
    return protein, ligand


def process_ligand(ligand, res_name):
    """
    Add bond orders to a pdb ligand
    1. Select the ligand component with name "res_name"
    2. Get the corresponding SMILES from pypdb
    3. Create a template molecule from the SMILES in step 2
    4. Write the PDB file to a stream
    5. Read the stream into an RDKit molecule
    6. Assign the bond orders from the template from step 3
    :param ligand: ligand as generated by prody
    :param res_name: residue name of ligand to extract
    :return: molecule with bond orders assigned
    """
    output = StringIO()
    sub_mol = ligand.select(f"resname {res_name}")
    chem_desc = describe_chemical(f"{res_name}")
    sub_smiles = chem_desc["describeHet"]["ligandInfo"]["ligand"]["smiles"]
    template = AllChem.MolFromSmiles(sub_smiles)
    writePDBStream(output, sub_mol)
    pdb_string = output.getvalue()
    rd_mol = AllChem.MolFromPDBBlock(pdb_string)
    new_mol = AllChem.AssignBondOrdersFromTemplate(template, rd_mol)
    return new_mol


def write_pdb(protein, pdb_name):
    """
    Write a prody protein to a pdb file
    :param protein: protein object from prody
    :param pdb_name: base name for the pdb file
    :return: None
    """
    output_pdb_name = f"{pdb_name}_protein.pdb"
    writePDB(f"{output_pdb_name}", protein)
    print(f"wrote {output_pdb_name}")


def write_sdf(new_mol, pdb_name, res_name):
    """
    Write an RDKit molecule to an SD file
    :param new_mol:
    :param pdb_name:
    :param res_name:
    :return:
    """
    outfile_name = f"{pdb_name}_{res_name}_ligand.sdf"
    writer = Chem.SDWriter(f"{outfile_name}")
    writer.write(new_mol)
    print(f"wrote {outfile_name}")


def main(pdb_name):
    """
    Read Ligand Expo data, split pdb into protein and ligands,
    write protein pdb, write ligand sdf files
    :param pdb_name: id from the pdb, doesn't need to have an extension
    :return:
    """
    protein, ligand = get_pdb_components(pdb_name)
    write_pdb(protein, pdb_name)

    res_name_list = list(set(ligand.getResnames()))
    for res in res_name_list:
        new_mol = process_ligand(ligand, res)
        write_sdf(new_mol, pdb_name, res)


# if __name__ == "__main__":
#     if len(sys.argv) == 2:
#         main(sys.argv[1])
#     else:
#         print("Usage: {sys.argv[1]} pdb_id", file=sys.stderr)

In [7]:
import os

from Bio.PDB import PDBParser, PDBIO, Select


def is_het(residue):
    res = residue.id[0]
    return res != " " and res != "W"


class ResidueSelect(Select):
    def __init__(self, chain, residue):
        self.chain = chain
        self.residue = residue

    def accept_chain(self, chain):
        return chain.id == self.chain.id

    def accept_residue(self, residue):
        """ Recognition of heteroatoms - Remove water molecules """
        return residue == self.residue and is_het(residue)


def extract_ligands(pdb_file):
    """ Extraction of the heteroatoms of .pdb files """

    i = 0
    pdb = PDBParser().get_structure('mpro', pdb_file)
    io = PDBIO()
    io.set_structure(pdb)
    for model in pdb:
        for chain in model:
            for residue in chain:
                if not is_het(residue):
                    continue
                print(f"saving {chain} {residue}")
                io.save(f"lig_{i}.pdb", ResidueSelect(chain, residue))
                i += 1

In [8]:
extract_ligands('/Users/williammccorkindale/Downloads/PhD_data/FRESCO/Apo_Mpro_6YB7/6lu7.pdb')

saving <Chain id=C> <Residue 02J het=H_02J resseq=1 icode= >
saving <Chain id=C> <Residue PJE het=H_PJE resseq=5 icode= >
saving <Chain id=C> <Residue 010 het=H_010 resseq=6 icode= >


/Users/williammccorkindale/miniconda3/envs/csd_env/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 2811.
  PDBConstructionWarning,


In [26]:
from ccdc.docking import Docker
from ccdc.io import MoleculeReader, EntryReader

docker = Docker()
settings = docker.settings

# mpro_protein_file = '/Users/williammccorkindale/Downloads/PhD_data/FRESCO/Apo_Mpro_6YB7/6YB7_model.pdb'
# settings.add_protein_file(mpro_protein_file)

mpro_dir = '/Users/williammccorkindale/Downloads/PhD_data/FRESCO/Apo_Mpro_6YB7'

# mpro_native_ligand_file = f'/Users/williammccorkindale/Downloads/PhD_data/FRESCO/Apo_Mpro_6YB7/6lu7.pdb'

mpro_protein_file = f'{mpro_dir}/6LU7_protein.mol2'
settings.add_protein_file(mpro_protein_file)

mpro_native_ligand_file = f'{mpro_dir}/6LU7_ligand.mol2'
mpro_native_ligand = MoleculeReader(mpro_native_ligand_file)[0]
print(mpro_native_ligand.smiles)
mpro_protein = settings.proteins[0]
settings.binding_site = settings.BindingSiteFromLigand(mpro_protein, mpro_native_ligand, 8.0)

settings.fitness_function = 'plp'
settings.autoscale = 10.
settings.early_termination = False

import tempfile
batch_tempd = tempfile.mkdtemp()
settings.output_directory = '/Users/williammccorkindale/ml_physics/smi2wyck/notebooks/docking/results/'
settings.output_file = 'docked_ligands.mol2'

# mpro_ligand_file = '/Applications/CCDC/Python_API_2022/docs/example_files/SAH.mol2'
mpro_ligand = MoleculeReader(mpro_native_ligand_file)[0]
settings.add_ligand_file(mpro_native_ligand_file, 5)

results = docker.dock() 
print(results.return_code)


CC(C)CC(NC(=O)C(NC(=O)C(C)NC(=O)C1=NOC(=C1)C)C(C)C)C(=O)NC(CC1CCNC1=O)[C]=CC(=O)OCc1ccccc1
Starting GOLD with conf file /Users/williammccorkindale/ml_physics/smi2wyck/notebooks/docking/api_gold.conf
Setting up GOLD environment...
GOLD Version 2022.1.0
Running:
 
     "/Applications/CCDC/Discovery_2022/GOLD/gold/d_macx/bin/gold_macx" "/Users/williammccorkindale/ml_physics/smi2wyck/notebooks/docking/api_gold.conf"

1


In [27]:
ligands = results.ligands

In [30]:
print(ligands[0].fitness())

85.4826
